# Testing Squiggy Installation from TestPyPI

This notebook helps you verify that Squiggy was installed correctly from TestPyPI.

**Prerequisites:**
- You have installed squiggy-positron from TestPyPI
- You are running this in Positron IDE with Python 3.12+

**Test data is bundled with the package** - no additional downloads needed!

Run each cell in order and verify the outputs match the expected results.

## Test 1: Import and Version Check

**Expected:** Version number should print without errors

In [ ]:
import squiggy

print(f"✓ Squiggy version: {squiggy.__version__}")
print(f"✓ Package location: {squiggy.__file__}")

## Test 2: Verify Dependencies

**Expected:** All dependencies should import successfully

In [ ]:
import pod5
import pysam
import bokeh
import numpy

print("✓ pod5 version:", pod5.__version__)
print("✓ pysam version:", pysam.__version__)
print("✓ bokeh version:", bokeh.__version__)
print("✓ numpy version:", numpy.__version__)
print("\n✓ All dependencies imported successfully!")

## Test 3: Check API Surface

**Expected:** All major classes and enums should be available

In [ ]:
from squiggy import Pod5File, BamFile, FastaFile
from squiggy import NormalizationMethod, PlotMode, Theme

print("✓ OO API classes available")

from squiggy import load_pod5, load_bam, plot_read, close_pod5, close_bam

print("✓ Legacy API functions available")

print(f"\nPlot modes: {[m.value for m in PlotMode]}")
print(f"Normalization methods: {[n.value for n in NormalizationMethod]}")
print(f"Themes: {[t.value for t in Theme]}")

## Test 4: Access Bundled Test Data

**Expected:** Test data files should be accessible via `get_test_data_path()`

In [ ]:
# Get paths to bundled test files
pod5_path = squiggy.get_test_data_path('yeast_trna_reads.pod5')
bam_path = squiggy.get_test_data_path('yeast_trna_mappings.bam')

print(f"✓ POD5 test file: {pod5_path}")
print(f"✓ BAM test file: {bam_path}")

# List all available test files
import os
data_dir = squiggy.get_test_data_path()
test_files = os.listdir(data_dir)
print(f"\n✓ Available test files: {', '.join(test_files)}")

## Test 5: Load POD5 File

**Expected:** POD5 file should load and show 180 reads

In [ ]:
pod5 = squiggy.Pod5File(pod5_path)

print(f"✓ Loaded POD5 file: {pod5.path}")
print(f"✓ Number of reads: {len(pod5)}")
print(f"✓ First 5 read IDs:")
for read_id in pod5.read_ids[:5]:
    print(f"  - {read_id}")

## Test 6: Access Read Data

**Expected:** Read should have signal data and metadata

In [ ]:
# Get the first read
read = pod5.get_read(pod5.read_ids[0])

print(f"✓ Read ID: {read.read_id}")
print(f"✓ Signal samples: {len(read.signal):,}")
print(f"✓ Mean quality score: {read.mean_qscore:.2f}")
print(f"✓ Duration: {read.duration:.2f} seconds")
print(f"✓ Signal range: {read.signal.min():.1f} to {read.signal.max():.1f}")

## Test 7: Create a Simple Plot

**Expected:** A Bokeh figure object should be created and displayed

In [ ]:
from bokeh.plotting import show, output_notebook

# Enable Bokeh output in Jupyter
output_notebook()

# Create a plot
fig = read.plot(
    mode=PlotMode.SINGLE,
    normalization=NormalizationMethod.ZNORM,
    theme=Theme.LIGHT
)

print(f"✓ Created plot (type: {type(fig).__name__})")
print(f"✓ Plot title: {fig.title.text}")

# Display the plot
show(fig)

## Test 8: Test Different Normalization Methods

**Expected:** Each normalization method should produce a different plot

In [ ]:
# Test MEDIAN normalization
fig_median = read.plot(
    mode=PlotMode.SINGLE,
    normalization=NormalizationMethod.MEDIAN,
    theme=Theme.LIGHT
)
print("✓ MEDIAN normalization plot created")

# Test MAD normalization
fig_mad = read.plot(
    mode=PlotMode.SINGLE,
    normalization=NormalizationMethod.MAD,
    theme=Theme.LIGHT
)
print("✓ MAD normalization plot created")

# Test NONE (raw signal)
fig_raw = read.plot(
    mode=PlotMode.SINGLE,
    normalization=NormalizationMethod.NONE,
    theme=Theme.LIGHT
)
print("✓ RAW signal plot created")

print("\n✓ All normalization methods work!")

## Test 9: Load BAM File

**Expected:** BAM file should load and show alignment information

In [ ]:
bam = squiggy.BamFile(bam_path)

print(f"✓ Loaded BAM file: {bam.path}")
print(f"✓ References found:")
for ref_name, read_ids in list(bam.references.items())[:5]:
    print(f"  - {ref_name}: {len(read_ids)} reads")

print(f"\n✓ Total aligned reads: {sum(len(reads) for reads in bam.references.values())}")

## Test 10: Test Alignment Integration

**Expected:** Should be able to get alignment data for aligned reads and create event-aligned plots

In [ ]:
# Find an aligned read
aligned_read = None
for read_id in pod5.read_ids[:10]:
    test_read = pod5.get_read(read_id)
    alignment = test_read.get_alignment(bam)
    if alignment:
        aligned_read = test_read
        break

if aligned_read:
    alignment = aligned_read.get_alignment(bam)
    print(f"✓ Found aligned read: {aligned_read.read_id}")
    print(f"✓ Aligned to: {alignment.chromosome}")
    print(f"✓ Position: {alignment.start:,}-{alignment.end:,}")
    print(f"✓ Strand: {alignment.strand}")
    print(f"✓ Bases called: {len(alignment.bases)}")
    
    # Create event-aligned plot
    fig_eventalign = aligned_read.plot(
        mode=PlotMode.EVENTALIGN,
        normalization=NormalizationMethod.ZNORM,
        bam_file=bam,
        theme=Theme.LIGHT
    )
    print("\n✓ Event-aligned plot created!")
    show(fig_eventalign)
else:
    print("⚠ No aligned reads found in first 10 reads")

## Test 11: Cleanup

**Expected:** Files should close without errors

In [ ]:
pod5.close()
print("✓ POD5 file closed")

bam.close()
print("✓ BAM file closed")

print("\n✓ All tests completed successfully!")

---

## Summary

If all cells ran without errors, your Squiggy installation from TestPyPI is working correctly!

### What was tested:
- ✓ Package import and version
- ✓ All dependencies (pod5, pysam, bokeh, numpy)
- ✓ API surface (classes, functions, enums)
- ✓ **Bundled test data access** (no Git LFS needed!)
- ✓ POD5 file loading
- ✓ Read data access
- ✓ Basic plotting
- ✓ Multiple normalization methods
- ✓ BAM file loading
- ✓ Alignment integration
- ✓ Event-aligned plotting

### Next Steps:
- Install the Squiggy Positron extension from the marketplace
- Use the extension's UI to interactively explore your data
- Check out `notebook_api_demo.ipynb` for more advanced usage examples

### Test Data Location:
The test data is now permanently available at:
```python
squiggy.get_test_data_path('yeast_trna_reads.pod5')
squiggy.get_test_data_path('yeast_trna_mappings.bam')
```
No need to download anything separately - it's bundled with the package!